In [2]:
#Some Import Mumbo-Jumbo
from datascience import *
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

## Baseball Decision Making (Weighted On Base Average)

<img src="https://cdn.vox-cdn.com/thumbor/7XmqqP0HZbz07vPJZZud8Wc9m6w=/0x0:4098x2720/1200x800/filters:focal(1439x98:2093x752)/cdn.vox-cdn.com/uploads/chorus_image/image/54784663/usa_today_10033424.0.jpg" style="width: 500px; height: 350px;" />

In [6]:
woba_stats = Table().read_table('retrosheet-events-plus-woba-2005_2015.csv')
woba_stats

game_id,year_id,away_team_id,home_team_id,bat_id,pit_id,bat_home_id,event_cd,wOBA_pts,time_since_1900,inn_ct,outs_ct,away_score_ct,home_score_ct,bat_hand_cd,pit_hand_cd,base1_run_id,base2_run_id,base3_run_id,bat_fld_cd,bat_lineup_id,rbi_ct,battedball_cd,pitch_seq_tx,game_new_fl,game_end_fl,po1_fld_cd,po2_fld_cd,inn_new_fl,inn_end_fl,inn_runs_ct,game_pa_ct,inn_pa_ct,pit_start_fl,pa_ball_ct,pa_strike_ct,event_runs_ct,bat_fate_id,run1_fate_id,run2_fate_id,run3_fate_id
ANA200504050,2005,TEX,ANA,soria001,colob001,0,21,1.252,3.32171e+09,1,0,0,0,R,R,nan,nan,nan,4,1,0,L,BX,T,F,0,0,T,F,0,0,0,T,1,1,0,0,0,0,0
ANA200504050,2005,TEX,ANA,blalh001,colob001,0,14,0.703,3.32171e+09,1,0,0,0,L,R,nan,soria001,nan,5,2,0,nan,BBBB,F,F,0,0,F,F,0,1,1,T,4,0,0,0,0,0,0
ANA200504050,2005,TEX,ANA,younm003,colob001,0,2,0,3.32171e+09,1,0,0,0,R,R,blalh001,soria001,nan,6,3,0,G,CB*BFFX,F,F,6,0,F,F,0,2,2,T,2,4,0,0,0,0,0
ANA200504050,2005,TEX,ANA,teixm001,colob001,0,3,0,3.32171e+09,1,1,0,0,L,R,younm003,nan,soria001,3,4,0,nan,CFBFBT,F,F,2,0,F,F,0,3,3,T,2,4,0,0,0,0,0
ANA200504050,2005,TEX,ANA,hidar001,colob001,0,2,0,3.32171e+09,1,2,0,0,R,R,younm003,nan,soria001,9,5,0,F,FCX,F,F,7,0,F,T,0,4,4,T,0,3,0,0,0,0,0
ANA200504050,2005,TEX,ANA,erstd001,dresr001,1,14,0.703,3.32171e+09,1,0,0,0,L,R,nan,nan,nan,3,1,0,nan,BBBCCB,F,F,0,0,T,F,0,0,0,T,4,2,0,0,0,0,0
ANA200504050,2005,TEX,ANA,davaj001,dresr001,1,2,0,3.32171e+09,1,0,0,0,L,R,erstd001,nan,nan,10,2,0,G,1X,F,F,6,3,F,F,0,1,1,T,0,1,0,0,0,0,0
ANA200504050,2005,TEX,ANA,guerv001,dresr001,1,23,2.017,3.32171e+09,1,2,0,0,R,R,nan,nan,nan,9,3,1,F,BBX,F,F,0,0,F,F,0,2,2,T,2,1,1,4,0,0,0
ANA200504050,2005,TEX,ANA,andeg001,dresr001,1,20,0.89,3.32171e+09,1,2,0,1,L,R,nan,nan,nan,7,4,0,L,X,F,F,0,0,F,F,1,3,3,T,0,1,0,0,0,0,0
ANA200504050,2005,TEX,ANA,finls001,dresr001,1,2,0,3.32171e+09,1,2,0,1,L,R,andeg001,nan,nan,8,5,0,F,CX,F,F,7,0,F,T,1,4,4,T,0,2,0,0,0,0,0


In [4]:
batting_stats = Table().read_table('batting.csv')
batting_stats

playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
abercda01,1871,1,TRO,nan,1,4,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan
addybo01,1871,1,RC1,nan,25,118,30,32,6,0,0,13,8,1,4,0,nan,nan,nan,nan,nan
allisar01,1871,1,CL1,nan,29,137,28,40,4,5,0,19,3,1,2,5,nan,nan,nan,nan,nan
allisdo01,1871,1,WS3,nan,27,133,28,44,10,2,2,27,1,1,0,2,nan,nan,nan,nan,nan
ansonca01,1871,1,RC1,nan,25,120,29,39,11,3,0,16,6,2,2,1,nan,nan,nan,nan,nan
armstbo01,1871,1,FW1,nan,12,49,9,11,2,1,0,5,0,1,0,1,nan,nan,nan,nan,nan
barkeal01,1871,1,RC1,nan,1,4,0,1,0,0,0,2,0,0,1,0,nan,nan,nan,nan,nan
barnero01,1871,1,BS1,nan,31,157,66,63,10,9,0,34,11,6,13,1,nan,nan,nan,nan,nan
barrebi01,1871,1,FW1,nan,1,5,1,1,1,0,0,1,0,0,0,0,nan,nan,nan,nan,nan
barrofr01,1871,1,BS1,nan,18,86,13,13,2,1,0,11,1,0,0,0,nan,nan,nan,nan,nan


In [5]:
team_stats = Table().read_table('teams.csv')
team_stats

yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,DivWin,WCWin,LgWin,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,CS,HBP,SF,RA,ER,ERA,CG,SHO,SV,IPouts,HA,HRA,BBA,SOA,E,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
1871,nan,BS1,BNA,nan,3,31,nan,20,10,nan,nan,N,nan,401,1372,426,70,37,3,60,19,73,nan,nan,nan,303,109,3.55,22,1,3,828,367,2,42,23,225,nan,0.838,Boston Red Stockings,South End Grounds I,nan,103,98,BOS,BS1,BS1
1871,nan,CH1,CNA,nan,2,28,nan,19,9,nan,nan,N,nan,302,1196,323,52,21,10,60,22,69,nan,nan,nan,241,77,2.76,25,0,1,753,308,6,28,22,218,nan,0.829,Chicago White Stockings,Union Base-Ball Grounds,nan,104,102,CHI,CH1,CH1
1871,nan,CL1,CFC,nan,8,29,nan,10,19,nan,nan,N,nan,249,1186,328,35,40,7,26,25,18,nan,nan,nan,341,116,4.11,23,0,0,762,346,13,53,34,223,nan,0.814,Cleveland Forest Citys,National Association Grounds,nan,96,100,CLE,CL1,CL1
1871,nan,FW1,KEK,nan,7,19,nan,7,12,nan,nan,N,nan,137,746,178,19,8,2,33,9,16,nan,nan,nan,243,97,5.17,19,1,0,507,261,5,21,17,163,nan,0.803,Fort Wayne Kekiongas,Hamilton Field,nan,101,107,KEK,FW1,FW1
1871,nan,NY2,NNA,nan,5,33,nan,16,17,nan,nan,N,nan,302,1404,403,43,21,1,33,15,46,nan,nan,nan,313,121,3.72,32,1,0,879,373,7,42,22,227,nan,0.839,New York Mutuals,Union Grounds (Brooklyn),nan,90,88,NYU,NY2,NY2
1871,nan,PH1,PNA,nan,1,28,nan,21,7,nan,nan,Y,nan,376,1281,410,66,27,9,46,23,56,nan,nan,nan,266,137,4.95,27,0,0,747,329,3,53,16,194,nan,0.845,Philadelphia Athletics,Jefferson Street Grounds,nan,102,98,ATH,PH1,PH1
1871,nan,RC1,ROK,nan,9,25,nan,4,21,nan,nan,N,nan,231,1036,274,44,25,3,38,30,53,nan,nan,nan,287,108,4.3,23,1,0,678,315,3,34,16,220,nan,0.821,Rockford Forest Citys,Agricultural Society Fair Grounds,nan,97,99,ROK,RC1,RC1
1871,nan,TRO,TRO,nan,6,29,nan,13,15,nan,nan,N,nan,351,1248,384,51,34,6,49,19,62,nan,nan,nan,362,153,5.51,28,0,0,750,431,4,75,12,198,nan,0.845,Troy Haymakers,Haymakers' Grounds,nan,101,100,TRO,TRO,TRO
1871,nan,WS3,OLY,nan,4,32,nan,15,15,nan,nan,N,nan,310,1353,375,54,26,6,48,13,48,nan,nan,nan,303,137,4.37,32,0,0,846,371,4,45,13,217,nan,0.85,Washington Olympics,Olympics Grounds,nan,94,98,OLY,WS3,WS3
1872,nan,BL1,BLC,nan,2,58,nan,35,19,nan,nan,N,nan,617,2576,747,94,35,14,27,28,35,15,nan,nan,434,173,3.02,48,1,1,1545,566,3,63,0,432,nan,0.829,Baltimore Canaries,Newington Park,nan,106,102,BAL,BL1,BL1


## Pythagorean Theorem

In its most simple form, the Pythagorean theorem is used as a method of using two known pieces of information in order to gain a third. In mathematics, this applies to triangles. Meanwhile for baseball, we want to find a way to connect known pieces of data to determine an unknown data point, in our case the number of wins.

Bill James was able to find such an equation which is now often regarded as the pythagorean theorem for baseball.

<img src="https://latex.codecogs.com/gif.latex?Win&space;\,&space;\,&space;Percentage&space;=&space;\frac{Runs&space;\,&space;\,Scored\,^{2}}{Runs&space;\,&space;\,Scored\,^{2}&plus;Runs&space;\,&space;\,Allowed&space;\,^{2}}" title="Win \, \, Percentage = \frac{Runs \, \,Scored\,^{2}}{Runs \, \,Scored\,^{2}+Runs \, \,Allowed \,^{2}}" />

This formula has several desirable properties.

1. The predicted win percentage is always between 0 and 1.

2. An increase in runs scored increases predicted win percentage.

3. A decrease in runs allowed increases predicted win percentage.

<b>Question 1:</b> The Cleveland Indians scored 872 runs and allowed 780 in their 2006 season. Calculate Their Predicted win percentage for the entirety of the 2006 season

<b>Question 2:</b> The true winning percentage was 0.481. How does your answer compare? Do you think that the Pythagorean Theorem will generally produce an answer that is useful to statisticians?

<b>Question 3:</b> Find the year in which the difference between the estimated win percentage and true win percentage is the smallest.

The real magic of the Pythagorean Theorem comes from the fact that not only can you measure the effectivenes of a team, but you can also measure the contributions of individual players.

For instance, say you are considering how a trade may affect your team. It's easy to subtract away the runs scored and allowed by an individual player and replace that data with data for the player that is going to be traded. This would allow you to gain an estimate of how the addition or removal of a player will affect the overall season results for a baseball club.

## Runs Created

Although the pythagorean theorem is extremely helpful to our ability to estimate the success of a whole team, comparing individual players without the context of their team becomes a bit more complex as all players have very different contributions to their teams.

As a result, James created another formula, known as runs created.

<img src="https://latex.codecogs.com/gif.latex?Runs&space;\,&space;\,&space;Created&space;=&space;\frac{(Hits&plus;BBR&plus;HBP)X(TB)}{(AB&plus;BB&plus;HBP)}" title="Runs \, \, Created = \frac{(Hits+BBR+HBP)X(TB)}{(AB+BB+HBP)}" />

<b>Where:</b>

Hits = number of hits

BB = number of walks by balls (4 balls results in a walk)

HBP = number of walks due to being hit by pitch

TB = Total Bases where Total Bases = # of Singles + 2(# of Doubles) + 3(# of Triples) + 4(# of Home Runs)

AB = Total At Bats


<b>Question 4:</b> Although this method is usually very accurate in determining the value of runs created, it isn't necessarily true that the values for an entire team will tell you anything or correspond in any way to a specific player. Why is this the case?

<b>Question 5:</b> Calculate the value of Runs Created for Alex Rodriguez (NYA) of the New York Yankees in the 2007 and 2008 seasons. In which season was he a greater impact towards his organization?

Answer:

<b>Question 6:</b> Estimate the value of runs created for all players on the Minnesota Twins in 2005 and determine which player created the most runs for the Twins in the 2005 season

**Hint: TB is not given to us in the MLB Data, try to add a column to the table that allows you to quickly calculate this data and then use another column to calculate Runs Created**

## Linear Weights

"In business and science we often try to predict a given variable (called Y or the dependent variable) from a set of independent variables (x1, x2 , . . . xn). Usually we try to find weights B1, B2, . . . Bn and a constant that make the quantity 

<img src="https://latex.codecogs.com/gif.latex?Constant&space;&plus;&space;B_1X_1&space;&plus;B_2X_2&space;&plus;&space;...&space;&plus;&space;B_nX_n" title="Constant + B_1X_1 +B_2X_2 + ... + B_nX_n" />

a good predictor for the dependent variable.Statisticians call the search for the weights and constant that best predict Y running a multiple linear regression. Sabermetricians (people who apply math to baseball) call the weights Linear Weights" - Mathletics (Wayne L. Winston)

In order to use linear weights, we must have values for individual events, so that those events can be weighed. For instance, if you are told that a car has 10 gallons of gas and asked how far the car can travel, you will be unable to solve the problem if you do not have any information on how far the car can go using one gallon of gas.

Similarly, we then need to find a way to determine the values of an individual event in baseball to find its contribution to the overall data. In this case, we can use runs scored in a season as our independent variable and inputs such as 2B, 3B, HR, etc. as our dependent variables.

You have recently discussed run states, which explain in great detail the expected value of runs at each and every position possible during a baseball game. This concept underscores that not all events in baseball are created equal. Beyond this, knowing run states allows for decision making in certain instances of the game, such as whether or not to bunt, or sacrifice a play in order to try to score.

The concept of using linear weights generally applies regression to a set of data to extrapolate just how many runs can be attributed to an event. Tom Tango is perhaps best known for creating a system of linear weights to calculate the offensive effectiveness of a baseball team using his idea of Weighted on Base Averages (WoBA). The idea behind it is quite simple, compile all possible values that impact offensive efficiency into one metric, and use it as a comparison. Tango's equation is as follows:

<img src="https://latex.codecogs.com/gif.latex?wOBA&space;=&space;\frac{0.72*NIBB&space;&plus;&space;0.75*&space;HBP&space;&plus;&space;0.90*1B&space;&plus;0.92*RBOE&plus;1.24*2B&plus;1.56*3B&plus;1.95*HR}{PA}" title="wOBA = \frac{0.72*NIBB + 0.75* HBP + 0.90*1B +0.92*RBOE+1.24*2B+1.56*3B+1.95*HR}{PA}" />

**Where:**

NIBB = Non-intentional bases on balls

HBP = Hit by pitch

1B = Single

RBOE = Reached base on error

2B = Double

3B = Triple

HR = Home run

PA = Plate appearance

<b>Question 7: Run States</b> Using the woba_stats table, find how many data sets exist in which there was one player out and one player on first base, with no other players on base.

<b>Question 8: Run States</b> Now, find how many points were scored during those instances and divide this number by your answer in Question 7.

<b>Question 9: Run States</b> In words, what does the above number represent?

<b>Question 10: </b> Calculate the wOBA for the player who had the most RBIs in the 2003 season.